In [49]:
import numpy as np 
import torch 
import torch.nn.functional as F
from scipy.stats import truncnorm
from scipy.sparse import csr_matrix
import random


from networkx.algorithms import bipartite

In [50]:
def oracle(data, language="marginal", n_best_items=None):
    """
    Allocation oracle.
    :param data: tensor with bids (valuations) in auctions shaped as (batch_size, n_agents, n_items)
    :param language: bidding language, either
        'additive' for heterogenous goods and additive utilities,
        'marginal' for homogenous goods and marginally decreasing utilities
            (utility of a bundle is sum of item-wise utilities, s.t. each new item produces less utility) or
        'unit-demand' for heterogenous goods and unit demand (utility of a bundle is max of item-wise utilities)
        * 'hierarchical' for hierarchical bundles?
    :param n_best_items: number of items to allocate, the default is all items
    :return: allocation, i.e. binary tensor with same shape as valuation
    """

    size, n_participants, n_items = data.size()
    if n_best_items is None:
        n_best_items = n_items
    n_best_items = min(n_best_items, n_items)

    if language == "additive":
        allocation = torch.argmax(data, 1, True)
        allocation = F.one_hot(allocation)[:, 0].permute((0, 2, 1))
        allocation = topk_allocation(data, allocation, n_best_items)

    elif language == "marginal":
        allocation = torch.zeros((size, n_participants)).long()
        for i, auction in enumerate(data):
            auction_copy = auction.clone()
            v, idx = auction_copy[:, 0], torch.zeros((n_participants,)).long()
            for j in range(n_best_items):
                part = torch.argmax(v).item()
                idx[part] += 1
                if j != n_items - 1:
                    v[part] = (
                        auction_copy[part, idx[part]]
                        - auction_copy[part, idx[part] - 1]
                    )
            allocation[i] = idx
        allocation = torch.zeros(size, n_participants, n_items + 1).scatter_(
            2, allocation.unsqueeze(-1), 1
        )[:, :, 1:]

    elif language == "unit-demand":
        n_best_items = min(n_best_items, n_participants)
        allocation = torch.zeros(size, n_participants, n_items)
        for i, auction in enumerate(data):
            allocation_cur = torch.zeros(n_participants, n_items)
            graph = csr_matrix(-auction.detach().numpy())
            graph = bipartite.from_biadjacency_matrix(graph)
            matching = bipartite.matching.minimum_weight_full_matching(graph)
            for part in range(n_participants):
                if part in matching.keys():
                    item = matching[part] - n_participants
                    allocation_cur[part, item] = 1
            allocation[i] = allocation_cur
        allocation = topk_allocation(data, allocation, n_best_items)

    else:
        raise NotImplementedError(
            "Only 'additive', 'marginal', and 'unit-demand' languages are implemented"
        )

    return allocation


def topk_allocation(data, allocation, n_best_items):
    threshold = torch.topk(
        (data * allocation).reshape(data.shape[0], -1), n_best_items, -1
    )[0][:, -1].view(-1, 1, 1)
    allocation[data < threshold] = 0
    return allocation


def get_v(data, allocation):
    """
    :param data: tensor with bids (valuations) in auctions shaped as (batch_size, n_agents, n_items)
    :param allocation: efficient allocation, output of oracle, binary tensor shaped as (batch_size, n_agents, n_items)

    :return: total value of objects gained by each agent in each auction with respect to the efficient allocations,
        tensor shaped as (batch_size, n_agents)
    """
    return torch.sum(data * allocation, 2)


def delete_agent(x, i):
    mask = torch.ones(x.size()[1]).int()
    mask[i] = 0
    return x[:, mask.bool()]


def get_v_sum_but_i(v):
    return torch.cat(
        [torch.sum(delete_agent(v, i), dim=1).view(-1, 1) for i in range(v.size()[1])],
        dim=1,
    )

In [51]:
def VCG(batch, language="marginal"):
    """
    VCG efficient and truthful mechanism.
    :param batch: tensor with bids (valuations) in auctions shaped as (batch_size, n_agents, n_items)
    :return: VCG prices t, tensor shaped as (batch_size, n_agents)
    """
    allocation = oracle(batch, language)
    v = get_v(batch, allocation)
    v_sum_but_i = get_v_sum_but_i(v)

    h = []
    for i in range(batch.shape[1]):
        batch_cur = delete_agent(batch, i)
        allocation_cur = oracle(batch_cur, language)
        v_cur = get_v(batch_cur, allocation_cur)
        v_sum_cur = v_cur.sum(dim=-1).view(-1, 1)
        h.append(v_sum_cur)
    h = torch.cat(h, dim=1)

    t = h - v_sum_but_i
    return t

In [52]:
if __name__ == "__main__":
    n_auctions, n_agents, n_items = 100000, 7, 3
    x, y, z = n_auctions, n_agents, n_items
    
    def calc_data_rate():
        W  = 10**6 # bandwidth
        d_i = random.uniform(100, 200) # distance from user to BS
        p_i_trans = random.uniform(0.4, 0.8) # transmit power of user
        h_i = random.gauss(0, 1) # small-fading channel gain of user
        sigma = 10**-17 # noise variance 
        P_i = W * np.log2(1 + (p_i_trans * abs(h_i) * (d_i ** -2)) / (W * sigma))

        return P_i

    def gen_truncnorm(mean, std, a, b, lens): 
        a_trunc = (a - mean) / std
        b_trunc = (b - mean) / std

        trunc_samples = truncnorm.rvs(
        a_trunc, b_trunc, loc=mean, scale=std, size=lens)

        return trunc_samples

    P_vm = 10 ** 12 # processing power of VM
    N_D = 4 * 10 ** 8 # number of data points
    C_tot_flops = 13.31 * 10 ** 9 # total number of LDM FLOPs
    B_mem = 2304 * 10 ** 9 # memory bandwidth (RTX 4060)
    Q_thresh = 3840 * 2160 # threshold for resolution factor 

    gamma = 0.5
    phi_1 = 0.5
    phi_2 = 0.5

    T_req_val = 0.1
    Q_req_val = 2.0

    k = [0.5 for i in range(8)]

    def calc_comp_latency(file_size):
        ans = gamma * (phi_1 * (C_tot_flops / P_vm) + (1 + phi_2) * (file_size / B_mem))
        return ans

    tot_latencies = []
    tot_qualities = []
    val = []

    case_1, case_2, case_3, case_4 = [], [], [], []

    file_sizes      = np.loadtxt('../file_sizes.txt')
    res_factors     = np.loadtxt('../factors/res_factor.txt')
    noise_factors   = np.loadtxt("../factors/noise_factor.txt")
    quality_factors = np.loadtxt("../factors/quality_factor.txt")

    for i in range(x):
        for j in range(y):
            for res in range(z):
                idx = random.randint(0, len(file_sizes) - 1)
                file_size = file_sizes[idx]

                data_rate = calc_data_rate()

                trans_latency = file_size / data_rate
                comp_latency = calc_comp_latency(file_size)
                ret_latency = file_size / data_rate * random.uniform(0.8, 1.2)

                tot_latency = trans_latency + comp_latency + ret_latency

                tot_latencies.append(tot_latency)

                Q_max = max([noise_factors[idx], quality_factors[idx], res_factors[idx]])
                Q_min = min([noise_factors[idx], quality_factors[idx], res_factors[idx]])

                cur_noise = (noise_factors[idx] - Q_min) / (Q_max - Q_min)
                cur_quality = (quality_factors[idx] - Q_min) / (Q_max - Q_min)
                cur_resolution = (res_factors[idx] - Q_min) / (Q_max - Q_min)

                tot_qual = cur_noise + cur_quality + cur_resolution
                tot_qualities.append(tot_qual)

                T_req_i = gen_truncnorm(0, T_req_val, 0, 1, 1)[0]
                Q_req_i = random.uniform(0.0, Q_req_val)

                cur_val = 0

                if (tot_qual >= Q_req_i) and (tot_latency <= T_req_i):
                    cur_val = k[1] * (tot_qual - Q_req_i) + k[2] * (T_req_i - tot_latency) + k[3]
                    case_1.append(cur_val)
                elif (tot_qual >= Q_req_i) and (tot_latency > T_req_i):
                    cur_val = k[4] * (tot_qual - Q_req_i) + k[5]
                    case_2.append(cur_val)
                elif (tot_qual < Q_req_i) and (tot_latency <= T_req_i):
                    cur_val = k[6] * (T_req_i - tot_latency) + k[7]
                    case_3.append(cur_val)
                else:
                    cur_val = 0
                    case_4.append(cur_val)

                val.append(cur_val)

    print(np.array(val).shape)
    val = np.array(val)
    XX = val.reshape(x, y, z)

    batch = torch.from_numpy(XX)
    prices= VCG(batch, "marginal")
    prices = prices.detach().numpy()
    print("payoff", float(prices.sum()) / n_auctions)

/tmp/ipykernel_1039317/1297483777.py:73: RuntimeWarning: invalid value encountered in scalar divide
  cur_noise = (noise_factors[idx] - Q_min) / (Q_max - Q_min)


(2100000,)
payoff 1.8606773387390276


In [53]:
import matplotlib.pyplot as plt

In [71]:
prices.shape

(100000, 7)